# Getting started

**Adapted from**: SOSTOOLS' SOSDEMO1 (See Section 4.1 of [SOSTOOLS User's Manual](http://sysos.eng.ox.ac.uk/sostools/sostools.pdf)) and Example 2.4 of [PJ08]

P. Parrilo and A. Jadbabaie
*Approximation of the joint spectral radius using sum of squares*.
Linear Algebra and its Applications, Elsevier (2008), 428, 2385-2402

In [1]:
using DynamicPolynomials
@polyvar x y
p = 2*x^4 + 2*x^3*y - x^2*y^2 + 5*y^4

2x⁴ + 2x³y - x²y² + 5y⁴

We need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v0.21.6/installation/#Supported-solvers) for a list of the available choices.
We use `SOSModel` instead of `Model` to be able to use the `>=` syntax for Sum-of-Squares constraints.

In [2]:
using SumOfSquares
import CSDP
solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
model = SOSModel(solver)
con_ref = @constraint(model, p >= 0)
optimize!(model)
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

We see above that the solver found a feasible solution.
We now inspect this solution:

In [3]:
q = gram_matrix(con_ref)

GramMatrix{Float64, MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}, Float64, SymMatrix{Float64}}([2.0 1.0 -2.9518518518518517; 1.0 4.903703703703703 4.4408920985006264e-17; -2.9518518518518517 4.4408920985006264e-17 5.0], MonomialBasis{DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}}(DynamicPolynomials.Monomial{true}[x², xy, y²]))

We can get the SOS decomposition from the gram matrix as follows:

In [4]:
sosdec = SOSDecomposition(q)

(-1.4043287476933917*x^2 - 0.685602733872361*x*y + 2.123635092514818*y^2)^2 + (-0.02034325143230049*x^2 - 2.1053486943394346*x*y - 0.6931517747987983*y^2)^2 + (0.16567112157245217*x^2 - 0.03405099490006233*x*y + 0.098562725879792*y^2)^2

We now seek for the SOS decomposition of the following polynomial:

In [5]:
p = 4*x^4*y^6 + x^2 - x*y^2 + y^2

4x⁴y⁶ - xy² + x² + y²

We build the same model as previously with this new polynomial.
Here we can use `Model` instead of `SOSModel` as we explicitly constrain
`p` to belong to the SOS cone with `p in SOSCone()`.

In [6]:
model = Model(solver)
con_ref = @constraint(model, p in SOSCone())
optimize!(model)
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

We can query the SOS decomposition directly from the constraint reference
as follows:

In [7]:
sos_decomposition(con_ref)

(-1.9748040632461137*x^2*y^3 - 0.19297758710767504*x*y - 1.3159789811639922e-18*x + 0.8633544111907671*y)^2 + (-3.1407330732230084e-16*x^2*y^3 + 1.7452642425911877*x*y^2 - 1.915236642967106e-16*x*y - 0.7962855107812759*x - 8.047687522921668e-16*y)^2 + (0.2579336224358902*x^2*y^3 - 1.7639164207923265e-16*x*y^2 - 1.5486392128975754*x*y - 3.489133500379429e-17*x + 0.24383463418983603*y)^2 + (-6.207001556270809e-17*x^2*y^3 + 0.27599821010522474*x*y^2 - 7.853337855400115e-17*x*y + 0.6049209744419544*x - 1.5904569988781444e-16*y)^2 + (-0.18335527863613854*x^2*y^3 - 1.62761136620241e-16*x*y^2 - 0.1000963758981316*x*y - 3.490953648902543e-17*x - 0.4417735074073053*y)^2

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*